

# Multimodal Retrieval using Qdrant and Gemini



In [1]:
# Install the necessary requirements
%pip install git+https://github.com/run-llama/llama_index.git
%pip install ftfy regex tqdm
%pip install git+https://github.com/openai/CLIP.git
%pip install torch torchvision
%pip install matplotlib scikit-image
%pip install -U qdrant_client
%pip install wikipedia

  Cloning https://github.com/run-llama/llama_index.git to c:\users\spectra\appdata\local\temp\pip-req-build-rj9dvw8e
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/run-llama/llama_index.git 'C:\Users\Spectra\AppData\Local\Temp\pip-req-build-rj9dvw8e'
  fatal: the remote end hung up unexpectedly
  fatal: early EOF
  fatal: index-pack failed
  You can inspect what was checked out with 'git status'
  and retry with 'git restore --source=HEAD :/'

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/run-llama/llama_index.git 'C:\Users\Spectra\AppData\Local\Temp\pip-req-build-rj9dvw8e' did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/run-llama/llama_index.git 'C:\Users\Spectra\AppData\Local\Temp\pip-req-build-rj9dvw8e' did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subp

  Obtaining dependency information for ftfy from https://files.pythonhosted.org/packages/91/f8/dfa32d06cfcbdb76bc46e0f5d69c537de33f4cedb1a15cd4746ab45a6a26/ftfy-6.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for wcwidth<0.3.0,>=0.2.12 from https://files.pythonhosted.org/packages/fd/84/fd2ba7aafacbad3c4201d395674fc6348826569da3c0937e75505ead3528/wcwidth-0.2.13-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/53.4 kB ? eta -:--:--
   ----------------------- ---------------- 30.7/53.4 kB ? eta -:--:--
   ---------------------------------------- 53.4/53.4 kB 464.0 kB/s eta 0:00:00
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.5
    Uninstalling wcwidth-0.2.5:
      Successfully uninstalled wcwidth-0.2.5
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Spectra\AppData\Local\Temp\pip-req-build-2jy8fa0x'



  Cloning https://github.com/openai/CLIP.git to c:\users\spectra\appdata\local\temp\pip-req-build-2jy8fa0x
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/58/b8/51b956c2da9729390a3080397cd2f31171394543af7746681466e372f69a/torch-2.2.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/b5/56/38e892200f8638032b64f6fc8660049f0d00ccba086cf1dcb884bd6370d2/torchvision-0.17.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/198.6 MB 660.6 kB/s eta 0:05:01
   ---------------------------------------- 0.1/19

Note: you may need to restart the kernel to use updated packages.


  Obtaining dependency information for qdrant_client from https://files.pythonhosted.org/packages/98/c3/8b2cebb07e0258b478f770dc2e9d246be17318c3858673ace0b345d147b0/qdrant_client-1.7.3-py3-none-any.whl.metadata
  Obtaining dependency information for grpcio>=1.41.0 from https://files.pythonhosted.org/packages/7b/3e/a22e7a0ec6be5454dc540063ac5d0843eb72a4641a0892b54b16b1438c0a/grpcio-1.60.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for grpcio-tools>=1.41.0 from https://files.pythonhosted.org/packages/8c/1f/ffd7645afeaad43b141c80f997bba8509721a2bc674dfc0d710e63fb4a1a/grpcio_tools-1.60.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for portalocker<3.0.0,>=2.7.0 from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/206.3 kB ? eta -:--:--
   --------------------- ------------------ 112.6/206.3

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11707 sha256=988453f86c251064e8c532131b13210b94780861fac052d73abec88987f2ef01
  Stored in directory: c:\users\spectra\appdata\local\pip\cache\wheels\8f\ab\cb\45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [68]:
# Necessary imports
import os
import llama_index
from llama_index.indices.multi_modal.base import MultiModalVectorStoreIndex
from llama_index.vector_stores import QdrantVectorStore
from llama_index import SimpleDirectoryReader, StorageContext
from llama_index.embeddings.gemini import GeminiEmbedding
import qdrant_client
from llama_index import (
    SimpleDirectoryReader,
)
from llama_index import ServiceContext, PromptHelper
from llama_index.multi_modal_llms.gemini import GeminiMultiModal


In [17]:
# Set API Key
GOOGLE_API_KEY = "YOUR API KEY HERE"  
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [33]:
# Multimodal LLM
mllm = GeminiMultiModal(model_name="models/gemini-pro-vision")
# Using the embedding model to Gemini
embed_model = GeminiEmbedding(
    model_name="models/embedding-001", api_key=GOOGLE_API_KEY
)

## Build Multi-Modal index and Vector Store to index both text and images 

In [18]:


# Create a local Qdrant vector store
client = qdrant_client.QdrantClient(url="CLIENT URL",
                                    api_key="API KEY")

text_store = QdrantVectorStore(
    client=client, collection_name="text_collection"
)
image_store = QdrantVectorStore(
    client=client, collection_name="image_collection"
    
)

# Create storage context

storage_context = StorageContext.from_defaults(vector_store=text_store,image_store=image_store)

# Create the MultiModal index
documents = SimpleDirectoryReader("images").load_data()


In [28]:
# Set service context to your own preferences
prompt_helper = PromptHelper(
  context_window=4096, 
  num_output=256, 
  chunk_overlap_ratio=0.1, 
  chunk_size_limit=None
)

service_context = ServiceContext.from_defaults(
  llm=mllm,
  embed_model=embed_model,
  prompt_helper=prompt_helper
)

# Set the global service context
llama_index.set_global_service_context(service_context)

In [34]:
# Prepare index
index = MultiModalVectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context, embed_model=embed_model, llm=mllm)

c:\Users\Spectra\Desktop\price-discovery\.venv\Lib\site-packages\clip\clip.py:57: UserWarning: C:\Users\Spectra/.cache/clip\ViT-B-32.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
100%|████████████████████████████████████████| 338M/338M [08:57<00:00, 659kiB/s]


## Query the index using an image and a prompt

In [65]:
from llama_index.prompts import PromptTemplate


qa_tmpl_str = (
    "As a product price analyst, refer to the knowledge provided in the context and your own knowledge to generate an accurate and concise price range of a give product given its image and description. Return ONLY the price range of the product. The description is offered below:\n"
    "Prompt: {query_str}\n"
    "Response: "
)

qa_tmpl = PromptTemplate(qa_tmpl_str)

query_engine = index.as_query_engine(
    multi_modal_llm=mllm, image_qa_template=qa_tmpl,
    similarity_top_k=2, image_similarity_top_k=2
)

query_str = "Ascella X Softside Expandable Luggage with Spinners, Black, Checked-Large 29-Inch"
response = query_engine.image_query("images/B084P184MG.jpg", query_str)
print(response)

 $150.00 - $200.00


The actual price range for this product is 198.41